In [1]:
#pip install malaya
#pip install torch
#pip install PySastrawi
#import SparkSession
#import nltk
#from nltk.tokenize import word_tokenize
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, udf, regexp_replace
from pyspark.sql.types import StringType, ArrayType
from malaya import tokenizer
import malaya
#from nltk.tokenize import sent_tokenize
import torch
# Download NLTK punkt package (only needs to be done once)
#nltk.download('punkt')
#nltk.download('punkt_tab')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('averaged_perceptron_tagger_eng')

# Initialize SparkSession
spark = SparkSession.builder.appName("DataTransformationWithMalaya").getOrCreate()

# Create dummy data
data = [
    (1, "Saya suka makan nasi lemak di pagi hari."),
    (2, "Pelajar sedang belajar di perpustakaan universiti."),
    (3, "Makanan di restoran ini sangat lazat dan murah!"),
    (4, "Cuaca hari ini sangat panas dan berangin."),
    (5, "Komputer riba ini pantas tetapi baterinya cepat habis."),
    (6, "Pemandangan di pantai ini sangat cantik pada waktu petang."),
    (7, "Telefon bimbit ini tidak mahal, tetapi kualitinya sangat baik."),
    (8, "Kereta itu bergerak perlahan kerana jalan sesak."),
    (9, "Adakah kamu akan datang ke majlis perkahwinan minggu depan?"),
    (10, "Saya tidak tahu mengapa dia marah dengan keputusan itu."),
]

# Define schema and create DataFrame
columns = ["id", "text_column"]
df = spark.createDataFrame(data, schema=columns)

# Correct usage of lower() from pyspark.sql.functions
#df = df.withColumn("cleaned_text", lower(col("cleaned_text")))

# Show the DataFrame
df.show(truncate=False)

#from transformers import AutoTokenizer

# Use the fast tokenizer from Hugging Face
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)


# Print PyTorch version
# print(torch._version_)

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("DataTransformationWithMalaya") \
    .getOrCreate()

# Create DataFrame from dummy data
df = spark.createDataFrame(data, schema=columns)

# Text Preprocessing with Malaya
# Broadcast Variables
stopwords = set([
    'yang', 'dan', 'di', 'ke', 'dengan', 'pada', 'untuk', 'atau', 'itu', 'ada', 
    'ini', 'ini', 'saja', 'bukan', 'tersebut', 'oleh', 'kepada', 'di', 'sebuah'
])
stemmer = malaya.stem.sastrawi()
#pos_model = malaya.pos.crf()

broadcast_stopwords = spark.sparkContext.broadcast(stopwords)
broadcast_stemmer = spark.sparkContext.broadcast(stemmer)
#broadcast_pos_model = spark.sparkContext.broadcast(pos_model)

# Define your data transformation function
def run_data_transformation(df):
    ###########################
    # Step 1: Noise Removal
    df = df.withColumn("cleaned_text", regexp_replace("text_column", "[^a-zA-Z\\s]", ""))
    df = df.withColumn("cleaned_text", lower(col("cleaned_text")))

    # # Step 2: Tokenization
    
    # def nltk_tokenize(text):
    #     # Ensure text is a valid string
    #     if isinstance(text, str):
    #         return word_tokenize(text)  # Use NLTK word_tokenize
    #     else:
    #         return []
    
    # tokenize_udf = udf(nltk_tokenize, ArrayType(StringType()))
    # df = df.withColumn("tokens", tokenize_udf(col("cleaned_text")))
    def malaya_tokenize(text):
    # Ensure text is a valid string
        tokenizerInt = tokenizer.Tokenizer() 
        if isinstance(text, str):
            return tokenizerInt.tokenize(text)  # Use Malaya's tokenizer
        else:
            return []

    # Define a UDF using Malaya tokenizer
    tokenize_udf = udf(malaya_tokenize, ArrayType(StringType()))
    
    # Apply the UDF to the DataFrame
    df = df.withColumn("tokens", tokenize_udf(col("cleaned_text")))
    

    stopwords = broadcast_stopwords.value
    # Step 3: Stopword Removal using Broadcasted Stopwords
    def remove_stopwords(tokens):
        return [word for word in tokens if word not in stopwords]

    remove_stopwords_udf = udf(remove_stopwords, ArrayType(StringType()))
    df = df.withColumn("filtered_tokens", remove_stopwords_udf(col("tokens")))

    stemmer = broadcast_stemmer.value
    # Step 4: Stemming with Broadcasted Stemmer
    def malaya_stem(tokens):
        return [stemmer.stem(token) for token in tokens]

    stem_udf = udf(malaya_stem, ArrayType(StringType()))
    df = df.withColumn("stemmed_tokens", stem_udf(col("filtered_tokens")))

    # Step 5: POS Tagging with Broadcasted POS Model
    # def nltk_pos_tag(tokens):
    #     if isinstance(tokens, list):
    #         return nltk.pos_tag(tokens)  # Returns a list of (word, POS) tuples
    #     return []

    # pos_udf = udf(nltk_pos_tag, ArrayType(ArrayType(StringType())))
    # df = df.withColumn("pos_tags", pos_udf(col("tokens")))

    # # Step 6: Sentence Segmentation
    # def segment_sentences(text):
    #     # Ensure the input is a string
    #     if isinstance(text, str):
    #         return sent_tokenize(text)
    #     return []

    # segment_udf = udf(segment_sentences, ArrayType(StringType()))
    # df = df.withColumn("sentences", segment_udf(col("cleaned_text")))

    return df


spark = SparkSession.builder.appName("DataTransformationWithMalaya").getOrCreate()

# Step 4: Create a DataFrame from your data
df = spark.createDataFrame(data, schema=columns)

# Apply the transformation to the DataFrame
df_transformed = run_data_transformation(df)

# Show the transformed DataFrame
# Stop SparkSession
spark.stop()

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/12 09:46:02 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/12 09:46:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/12 09:46:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java

+---+--------------------------------------------------------------+
|id |text_column                                                   |
+---+--------------------------------------------------------------+
|1  |Saya suka makan nasi lemak di pagi hari.                      |
|2  |Pelajar sedang belajar di perpustakaan universiti.            |
|3  |Makanan di restoran ini sangat lazat dan murah!               |
|4  |Cuaca hari ini sangat panas dan berangin.                     |
|5  |Komputer riba ini pantas tetapi baterinya cepat habis.        |
|6  |Pemandangan di pantai ini sangat cantik pada waktu petang.    |
|7  |Telefon bimbit ini tidak mahal, tetapi kualitinya sangat baik.|
|8  |Kereta itu bergerak perlahan kerana jalan sesak.              |
|9  |Adakah kamu akan datang ke majlis perkahwinan minggu depan?   |
|10 |Saya tidak tahu mengapa dia marah dengan keputusan itu.       |
+---+--------------------------------------------------------------+

